<a href="https://colab.research.google.com/github/MaryaMalyshkina/Grand_line/blob/main/classif_model_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка среды

In [1]:
!pip install torch torchvision Pillow -q

In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.8 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Загрузка данных

In [4]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
model = YOLO("yolo11s-cls.pt")

In [6]:
print(model.model)  # архитектура модели
print(model.task)


ClassificationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )


In [7]:
results = model.train(
    data="/content/drive/MyDrive/defect_classification",
    epochs=100,           # количество эпох
    imgsz=224,            # размер изображения
    batch=16,             # размер батча
    lr0=0.01,             # начальный LR
    momentum=0.9,         # параметр оптимизатора
    weight_decay=5e-4,    # регуляризация
    patience=30,          # ранняя остановка
    save_period=10,       # сохранять чекпойнт каждые 10 эпох
    project="runs/classify_yolo_224", # папка для логов и весов
    name="yolo11s_6class_224px",          # имя эксперимента
    hsv_h=0.5, hsv_s=0.5, hsv_v=0.5,  # изменение цвета
    degrees=10, translate=0.1, scale=0.5  # геометрические трансформации
)

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/defect_classification, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.5, hsv_s=0.5, hsv_v=0.5, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.9, mosaic=1.0, multi_scale=0.0, name=yolo11s_6class_224px, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30

In [8]:
metrics = model.val()  # вычисляет accuracy, precision, recall, F1 и др.
print(metrics.results_dict)  # полный словарь метрик


Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-cls summary (fused): 47 layers, 5,441,814 parameters, 0 gradients, 12.0 GFLOPs
train: /content/drive/MyDrive/defect_classification/train... found 614 images in 6 classes ✅ 
val: /content/drive/MyDrive/defect_classification/val... found 116 images in 6 classes ✅ 
test: /content/drive/MyDrive/defect_classification/test... found 26 images in 6 classes ✅ 
val: Fast image access ✅ (ping: 4.2±3.9 ms, read: 22.2±19.4 MB/s, size: 160.7 KB)
val: Scanning /content/drive/MyDrive/defect_classification/val... 116 images, 0 corrupt: 100% ━━━━━━━━━━━━ 116/116 32.4Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 8/8 1.1s/it 8.9s
                   all      0.586      0.974
Speed: 0.9ms preprocess, 3.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val
{'metrics/accuracy_top1': 0.5862069129943848, 'metrics/accuracy_top5': 0.9741379022598267, 'f

In [10]:
model = YOLO("yolo11s-cls.pt")
results = model.train(
    data="/content/drive/MyDrive/defect_classification",
    epochs=300,
    imgsz=384,
    batch=64,
    lr0=0.001,
    lrf=0.01,
    optimizer='AdamW',
    weight_decay=0.0005,
    patience=50,
    pretrained=True,
    dropout=0.1,
    mixup=0.5,
    copy_paste=0.3,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=20.0,
    translate=0.2,
    scale=0.9,
    shear=2.0,
    perspective=0.0001,
    fliplr=0.5,
    mosaic=0.0,
    close_mosaic=0,
    auto_augment='randaugment',
    erasing=0.4,
    project="runs/classify_yolo_384",
    name="yolo11s_6class_384px",
    exist_ok=True,
    plots=True,
    save=True,
    amp=True,
)

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/defect_classification, degrees=20.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.5, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=0.0, multi_scale=0.0, name=yolo11s_6class_384px, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patie

In [11]:
metrics = model.val()  # вычисляет accuracy, precision, recall, F1 и др.
print(metrics.results_dict)  # полный словарь метрик

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-cls summary (fused): 47 layers, 5,441,814 parameters, 0 gradients, 12.0 GFLOPs
train: /content/drive/MyDrive/defect_classification/train... found 614 images in 6 classes ✅ 
val: /content/drive/MyDrive/defect_classification/val... found 116 images in 6 classes ✅ 
test: /content/drive/MyDrive/defect_classification/test... found 26 images in 6 classes ✅ 
val: Fast image access ✅ (ping: 2.2±2.0 ms, read: 12.0±3.0 MB/s, size: 160.7 KB)
val: Scanning /content/drive/MyDrive/defect_classification/val... 116 images, 0 corrupt: 100% ━━━━━━━━━━━━ 116/116 30.4Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 8/8 2.8s/it 22.6s
                   all      0.612      0.974
Speed: 2.9ms preprocess, 6.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val2
{'metrics/accuracy_top1': 0.6120689511299133, 'metrics/accuracy_top5': 0.9741379022598267, '

In [12]:
import shutil
# Сохранение результатов обучения
source_dir = "/content/runs/classify/runs/classify_yolo_384"
destination_dir = "/content/drive/MyDrive/runs_classif_yolo_backup"

# Если папка уже существует, удаляем её
if os.path.exists(destination_dir):
    shutil.rmtree(destination_dir)

# Копируем всю папку с содержимым
shutil.copytree(source_dir, destination_dir)

print(f" Папка успешно скопирована в: {destination_dir}")

 Папка успешно скопирована в: /content/drive/MyDrive/runs_classif_yolo_backup
